In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=8b84665079fc3c2376accc0d9e47d60f39fe44c063e5c49a5a9f7422f6e1f09a
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=65ec6c28e577656294d7ea4cfc9c1a7a687d43ceebc56a7fe44a6012d442fe2e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/btcusdt_3m.csv')
data.columns = [column.capitalize() for column in data.columns]

In [ ]:
data.dtypes

Datetime     object
Open        float64
High        float64
Low         float64
Close       float64
Volume      float64
dtype: object

In [ ]:
data

,Datetime,Open,High,Low,Close,Volume
0,2018-01-01 05:30:00,13715.65,13715.65,13666.11,13680.00,6.303815
1,2018-01-01 05:33:00,13679.98,13679.98,13554.44,13568.00,33.709206
2,2018-01-01 05:36:00,13568.00,13575.00,13501.01,13575.00,29.645987
3,2018-01-01 05:39:00,13539.99,13575.00,13400.01,13515.26,30.696039
4,2018-01-01 05:42:00,13515.26,13568.66,13473.47,13556.15,23.260966
...,...,...,...,...,...,...
703892,2022-01-12 05:18:00,42745.17,42745.17,42685.18,42687.42,23.505440
703893,2022-01-12 05:21:00,42687.43,42692.50,42633.97,42639.50,41.196100
703894,2022-01-12 05:24:00,42639.50,42697.36,42639.50,42697.36,31.459880
703895,2022-01-12 05:27:00,42697.36,42739.92,42672.09,42729.29,42.499600


In [ ]:
import pandas as pd

def EMA(values, n):
  return pd.Series(values).ewm(span = n).mean()

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return close.ewm(span=n, adjust=False).mean()

class EmaCrossStrategy(Strategy):
    n1 = 12
    n2 = 26
    n3 = 7

    def init(self):
        self.ema12 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema26 = self.I(EMA_Backtesting, self.data.Close, self.n2)
        self.ema7 = self.I(EMA_Backtesting, self.data.Close, self.n3)

    def next(self):
        if crossover(self.ema12, self.ema26) and self.data.Close<self.ema7:
            self.buy()
        elif self.data.Close>self.ema7:
            self.sell()

In [ ]:
from backtesting import Backtest
bt = Backtest(data, EmaCrossStrategy, cash=100000, commission=0.002,  trade_on_close=True)
stats = bt.run()
bt.plot()
stats

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the fi

Start                     2018-01-01 05:30:00
End                       2022-01-12 05:30:00
Duration                   1472 days 00:00:00
Exposure Time [%]                   33.489843
Equity Final [$]                          0.0
Equity Peak [$]                  408461.57838
Return [%]                             -100.0
Buy & Hold Return [%]              212.968056
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]             3706.445938
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                   -3.486373
Max. Drawdown Duration     1123 days 08:09:00
Avg. Drawdown Duration        7 days 15:58:00
# Trades                                   38
Win Rate [%]                        31.578947
Best Trade [%]                      52.857838
Worst Trade [%]                    -99.691988
Avg. Trade [%]                    